In [30]:
import numpy as np
from transformers import BertTokenizer

In [31]:
with open('corpus.txt', 'r') as fil:
    inp_data = fil.read()

In [32]:
def get_tokens(seq):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(seq))

def get_seq(ids):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    return tokenizer.convert_ids_to_tokens(ids)

print(inp_data[:25])
tkn = get_tokens(inp_data[:25])
seq = get_seq(tkn)
print(tkn)
print(seq)


i stand here i feel empty
[1045, 3233, 2182, 1045, 2514, 4064]
['i', 'stand', 'here', 'i', 'feel', 'empty']


In [33]:
trf_data = get_tokens(inp_data)
trf_data[:10]

[1045, 3233, 2182, 1045, 2514, 4064, 1037, 2465, 2695, 4175]

In [34]:
lst_data = get_seq(trf_data)
lst_data[:10]

['i', 'stand', 'here', 'i', 'feel', 'empty', 'a', 'class', 'post', 'count']

In [35]:
max_tkn = max(trf_data) + 1
coeff_mat = np.zeros((max_tkn, max_tkn), dtype=np.int64)

In [7]:
for t1, t2 in zip(trf_data, trf_data[1:]):
    coeff_mat[t1, t2] += 1

In [8]:
coeff_mat.sum(1, keepdims=True)

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [11]:
np.maximum(coeff_mat - 0.75, 0)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
coeff_mat = coeff_mat + 1
norm_mat = coeff_mat/coeff_mat.sum(1, keepdims=True)
norm_mat.shape 

(29583, 29583)

In [14]:
norm_mat

array([[3.38031978e-05, 3.38031978e-05, 3.38031978e-05, ...,
        3.38031978e-05, 3.38031978e-05, 3.38031978e-05],
       [3.38031978e-05, 3.38031978e-05, 3.38031978e-05, ...,
        3.38031978e-05, 3.38031978e-05, 3.38031978e-05],
       [3.38031978e-05, 3.38031978e-05, 3.38031978e-05, ...,
        3.38031978e-05, 3.38031978e-05, 3.38031978e-05],
       ...,
       [3.38031978e-05, 3.38031978e-05, 3.38031978e-05, ...,
        3.38031978e-05, 3.38031978e-05, 3.38031978e-05],
       [3.38031978e-05, 3.38031978e-05, 3.38031978e-05, ...,
        3.38031978e-05, 3.38031978e-05, 3.38031978e-05],
       [3.38020552e-05, 3.38020552e-05, 3.38020552e-05, ...,
        3.38020552e-05, 3.38020552e-05, 3.38020552e-05]])

In [13]:
p  = coeff_mat[1045]
p = p/p.sum()
sample = np.random.choice(len(p), p=p)

In [104]:
def genrate(seq_ln):
    idx = 1045
    for i in range(seq_ln):
        p  = norm_mat[idx]
        idx = np.random.choice(len(p), p=p)
        print(get_seq(idx))
genrate(10)

may
have
never
done
by
ad
##ore
leonard
cohen
is


In [56]:
from transformers import BertTokenizer

class BigramLM:
    def get_tokens(self, seq):
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(seq))

    def get_seq(self, ids):
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        return tokenizer.convert_ids_to_tokens(ids)
    
    def create_matrix(self, tokens):
        max_tkn = max(trf_data) + 1
        coeff_mat = np.zeros((max_tkn, max_tkn), dtype=np.int32)
        for t1, t2 in zip(tokens, tokens[1:]):
            coeff_mat[t1, t2] += 1
        return coeff_mat
    
    def __init__(self, data):
        self.token_data = self.get_tokens(data)
        self.coeff_mat = self.create_matrix(self.token_data)
    
    def genrate(self, seq_ln):
        idx = 1045
        for i in range(seq_ln):
            p  = coeff_mat[idx]
            p = p/p.sum()
            idx = np.random.choice(len(p), p=p)
            print(get_seq(idx))

        

In [61]:
with open('corpus.txt', 'r') as fil:
    inp_data = fil.read()
bgram = BigramLM(inp_data)

In [62]:
bgram.genrate(10)

begin
senator
senator
murderer
senator
servant
murderer
senator
serving
:


In [64]:
get_tokens("i")

[1045]

In [2]:
class BigramLM:
    def get_tokens(self, seq):
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(seq))

    def get_seq(self, ids):
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        return tokenizer.convert_ids_to_tokens(ids)
    
    def create_matrix(self, tokens):
        max_tkn = max(tokens) + 1
        coeff_mat = np.zeros((max_tkn, max_tkn), dtype=np.float64)
        for t1, t2 in zip(tokens, tokens[1:]):
            coeff_mat[t1, t2] += 1
        return coeff_mat/coeff_mat.sum(1, keepdims=True)
    
        
    def kneser_ney(self, tokens, d=0.75):
        coeff_mat = self.create_matrix(tokens)
        ugram = coeff_mat.sum(axis=1) / coeff_mat.sum()

        context_counts = np.sum(coeff_mat > 0, axis=0)
        discounted_counts = np.maximum(coeff_mat - d, 0)

        kneser_ney = np.zeros_like(coeff_mat, dtype=np.float64)
        for j in range(coeff_mat.shape[1]):
            kneser_ney[:, j] = np.where(context_counts[j] > 0, discounted_counts[:, j] / coeff_mat.sum(axis=1), ugram[j])
        return kneser_ney
    
    def __init__(self, data):
        self.token_data = self.get_tokens(data)
        self.coeff_mat = self.kneser_ney(self.token_data)
    
    def genrate(self, seq_ln, strt = 1045):
        idx = strt
        for i in range(seq_ln):
            p  = self.coeff_mat[idx]
            idx = np.random.choice(len(p), p=p)
            print(self.get_seq(idx))

        
with open('corpus.txt', 'r') as fil:
    inp_data = fil.read()
bgram = BigramLM(inp_data)
bgram.genrate(10)

/var/folders/jw/_f2zhz1d7cv9s1t6_f3s1dvc0000gn/T/ipykernel_76692/2553727299.py:15: RuntimeWarning: invalid value encountered in divide
  return coeff_mat/coeff_mat.sum(1, keepdims=True)


KeyboardInterrupt: 

In [42]:
from utils import emotion_scores

In [44]:
emotion_scores("i feel very happy today")

[{'label': 'sadness', 'score': 0.00022231916955206543},
 {'label': 'joy', 'score': 0.9990586638450623},
 {'label': 'love', 'score': 0.00032281389576382935},
 {'label': 'anger', 'score': 0.00011650829401332885},
 {'label': 'fear', 'score': 0.00012264464749023318},
 {'label': 'surprise', 'score': 0.00015699271170888096}]

In [5]:
emotion_scores("i feel")[1]['score']

0.9858515858650208

In [7]:
with open('corpus.txt', 'r') as fil:
    inp_data = fil.read().split("\n")
inp_data

['i stand here i feel empty a class post count link href http mooshilu',
 'i literally just text tychelle to see if she wants to hang out because reading what i just wrote about my nonexistent social life made me feel so pathetic',
 'i really feel regretful when hearing that shinae got married to another man oh it s really sad i really hope that alex and shinae can be a couple in real life they re perfect for each other',
 'i believed it was true love and feel devastated i wanted to settle down and have the whole marriage and kids thing with him',
 'i feel unimportant so inadequate',
 'i feel very low already',
 'i feel horrible they wrote again and again personifying an act they were not the cause of it was their progeny who should be genuflecting at her the wronged woman s feet',
 'ill just paraphrase i ranted about not being able to trust anybody and being hurt feeling rejected etc',
 'i feel aching at all times of day',
 'i feel so dumb when at first run through it all seems over m

In [8]:
tokens = set()
for sent in inp_data:
    sent = f'<s> {sent} <e>'
    tokens.update(sent.split())

In [5]:
token_to_index = {token: index for index, token in enumerate(tokens)}

In [100]:
import numpy as np
import pickle
from utils import emotion_scores


import warnings
warnings.filterwarnings("ignore")

class BigramLM:
    def get_tokens(self, seq):
        toks = seq.split()
        return [self.tok_id[tok] for tok in toks]
    
    def get_seq(self, ids):
        tokid = [self.id_tok[id] for id in ids]
        sent = " ".join(tokid)
        return sent
    
    def create_mat(self):
        tmp =  self.coeff_mat/self.coeff_mat.sum(1, keepdims=True)
        tmp += self.lov_mat
        return tmp/tmp.sum(1,keepdims = True)
    
    def laplace_mat(self):
        prob_mt = self.coeff_mat + 1
        return prob_mt/prob_mt.sum(1, keepdims=True)
    
    def kneser_ney_mat(self, d = 0.75):
        prob_mt = self.coeff_mat
        frst_trm = np.maximum(prob_mt - d, 0)/prob_mt.sum(1, keepdims=True)
        lmbd_trm = (d/prob_mt.sum(axis = 1,keepdims=True)) * np.count_nonzero(prob_mt, axis=1, keepdims=True)
        cont = np.count_nonzero(prob_mt, axis=0, keepdims=True)
        pcnt_trm = cont/ cont.sum()
        return frst_trm + lmbd_trm * pcnt_trm
    
    def genrate(self, size = -1):
        idx = self.get_tokens("<s>")[0]
        generated_sequence = ""
        i = 0
        while (self.get_seq([idx]) != "<e>"):
            if(i == size):
                generated_sequence += "..."
                break
            elif(i != 0):
                generated_sequence += self.get_seq([idx]) + " "
            p = self.prob_mat[idx]
            idx = int(np.random.choice(len(p), p=p))
            i += 1
        return generated_sequence
    
    def gen_matrix(self, tokens):
        max_tkn = max(tokens) + 1
        coeff_mat = np.zeros((max_tkn, max_tkn), dtype=np.float64)
        sad_mat = np.zeros((max_tkn, max_tkn), dtype=np.float64)
        joy_mat = np.zeros((max_tkn, max_tkn), dtype=np.float64)
        lov_mat = np.zeros((max_tkn, max_tkn), dtype=np.float64)
        ang_mat = np.zeros((max_tkn, max_tkn), dtype=np.float64)
        fea_mat = np.zeros((max_tkn, max_tkn), dtype=np.float64)
        sup_mat = np.zeros((max_tkn, max_tkn), dtype=np.float64)
        for t1, t2 in zip(tokens, tokens[1:]):
            coeff_mat[t1, t2] += 1
            if(coeff_mat[t1, t2] == 1):
                emote = emotion_scores(self.get_seq([t1,t2]))
                sad_mat[t1, t2] = emote[0]['score']
                joy_mat[t1, t2] = emote[1]['score']
                lov_mat[t1, t2] = emote[2]['score']
                ang_mat[t1, t2] = emote[3]['score']
                fea_mat[t1, t2] = emote[4]['score']
                sup_mat[t1, t2] = emote[5]['score']
        return coeff_mat, sad_mat, joy_mat, lov_mat, ang_mat, fea_mat, sup_mat

    def __init__(self, file, smoothing = 0):

        self.tokens = set()
        self.data = ""

        with open(file, 'r') as fil:
            data = fil.read()
        lin_data = data.split("\n")
        for sntc in lin_data:
            fsntc = f'<s> {sntc} <e> '
            self.data += fsntc
            self.tokens.update(fsntc.split())
        
        self.tok_id = {token: index for index, token in enumerate(self.tokens)}
        self.id_tok = {index: token for token, index in self.tok_id.items()}
        self.tokenized = self.get_tokens(self.data)

        self.coeff_mat, self.sad_mat, self.joy_mat, self.lov_mat, self.ang_mat, self.fea_mat, self.sup_mat = self.gen_matrix(self.tokenized)

        if(smoothing == 1):
            self.prob_mat = self.laplace_mat()
        elif(smoothing == 2):
            self.prob_mat = self.kneser_ney_mat()
        else:
            self.prob_mat = self.create_mat()

    def save(self, filename = "bigram_mdl.pkl"):
        with open(filename, 'wb') as f:
            pickle.dump(self, f)

    def get_emotion_score(self, size = -1):
        sntnc = self.genrate(size = size)
        emt_scr = emotion_scores(sntnc)
        emote = max(emt_scr, key=lambda x: x['score'])
        return sntnc, emote["label"], emote["score"]





def load(filename = "bigram_mdl.pkl"):
        with open(filename, 'rb') as f:
            obj = pickle.load(f)
        return obj

In [56]:
sent, score = bgram.get_emotion_score()
print(sent)

<s> i empathize with my divine enough that i feel i am feeling that master on toes


In [17]:
(bgram.coeff_mat).shape

(5431, 5431)

In [111]:
import pickle

def load(filename = "bigram_mdl.pkl"):
        with open(filename, 'rb') as f:
            obj = pickle.load(f)
        return obj

In [112]:
bgram = load()

ifeel


In [ ]:
from utils import emotion_scores

In [108]:
bgram.get_tokens("i love")

[7, 19]

In [109]:
bgram.prob_mat[7, 19]

0.0

In [110]:

emotion_scores("i love you")

[{'label': 'sadness', 'score': 0.004070514813065529},
 {'label': 'joy', 'score': 0.03258446604013443},
 {'label': 'love', 'score': 0.9608994722366333},
 {'label': 'anger', 'score': 0.0012419335544109344},
 {'label': 'fear', 'score': 0.0006723821279592812},
 {'label': 'surprise', 'score': 0.0005313107394613326}]

In [ ]:
import numpy as np
import pickle
from utils import emotion_scores


import warnings

warnings.filterwarnings("ignore")

np.random.seed(42)


class BigramLM:
    def get_tokens(self, seq):
        toks = seq.split()
        return [self.tok_id[tok] for tok in toks]

    def get_seq(self, ids):
        tokid = [self.id_tok[id] for id in ids]
        sent = " ".join(tokid)
        return sent

    def create_mat(self):
        return self.coeff_mat / self.coeff_mat.sum(1, keepdims=True)

    def laplace_mat(self):
        prob_mt = self.coeff_mat + 1
        return prob_mt / prob_mt.sum(1, keepdims=True)

    def kneser_ney_mat(self, d=0.75):
        prob_mt = self.coeff_mat
        frst_trm = np.maximum(prob_mt - d, 0) / prob_mt.sum(1, keepdims=True)
        lmbd_trm = (d / prob_mt.sum(axis=1, keepdims=True)) * np.count_nonzero(
            prob_mt, axis=1, keepdims=True
        )
        cont = np.count_nonzero(prob_mt, axis=0, keepdims=True)
        pcnt_trm = cont / cont.sum()
        return frst_trm + lmbd_trm * pcnt_trm

    def genrate(self, emotion, size=-1):
        idx = self.get_tokens("<s>")[0]
        generated_sequence = ""
        i = 0
        if emotion == "sad":
            mat = self.sad_mat
        elif emotion == "joy":
            mat = self.joy_mat
        elif emotion == "love":
            mat = self.lov_mat
        elif emotion == "angry":
            mat = self.ang_mat
        elif emotion == "fear":
            mat = self.fea_mat
        elif emotion == "surprise":
            mat = self.sup_mat
        else:
            mat = np.zeros_like(self.coeff_mat)

        curr_mat = mat
        curr_mat /= curr_mat.sum(1, keepdims=True)

        while self.get_seq([idx]) != "<e>":
            if i == size:
                generated_sequence += "..."
                break
            elif i != 0:
                generated_sequence += self.get_seq([idx]) + " "
            p = curr_mat[idx]
            idx = int(np.random.choice(len(p), p=p))
            i += 1
        return generated_sequence

    def gen_matrix(self, tokens):
        max_tkn = max(tokens) + 1
        coeff_mat = np.zeros((max_tkn, max_tkn), dtype=np.float64)
        sad_mat = np.zeros((max_tkn, max_tkn), dtype=np.float64)
        joy_mat = np.zeros((max_tkn, max_tkn), dtype=np.float64)
        lov_mat = np.zeros((max_tkn, max_tkn), dtype=np.float64)
        ang_mat = np.zeros((max_tkn, max_tkn), dtype=np.float64)
        fea_mat = np.zeros((max_tkn, max_tkn), dtype=np.float64)
        sup_mat = np.zeros((max_tkn, max_tkn), dtype=np.float64)
        for t1, t2 in zip(tokens, tokens[1:]):
            coeff_mat[t1, t2] += 1
            if coeff_mat[t1, t2] == 1:
                emote = emotion_scores(self.get_seq([t1, t2]))
                sad_mat[t1, t2] = emote[0]["score"]
                joy_mat[t1, t2] = emote[1]["score"]
                lov_mat[t1, t2] = emote[2]["score"]
                ang_mat[t1, t2] = emote[3]["score"]
                fea_mat[t1, t2] = emote[4]["score"]
                sup_mat[t1, t2] = emote[5]["score"]
        return coeff_mat, sad_mat, joy_mat, lov_mat, ang_mat, fea_mat, sup_mat

    def __init__(self, file, smoothing=0):

        self.tokens = set()
        self.data = ""

        with open(file, "r") as fil:
            data = fil.read()
        lin_data = data.split("\n")
        for sntc in lin_data:
            fsntc = f"<s> {sntc} <e> "
            self.data += fsntc
            self.tokens.update(fsntc.split())

        self.tok_id = {token: index for index, token in enumerate(self.tokens)}
        self.id_tok = {index: token for token, index in self.tok_id.items()}
        self.tokenized = self.get_tokens(self.data)

        (
            self.coeff_mat,
            self.sad_mat,
            self.joy_mat,
            self.lov_mat,
            self.ang_mat,
            self.fea_mat,
            self.sup_mat,
        ) = self.gen_matrix(self.tokenized)

        if smoothing == 1:
            self.prob_mat = self.laplace_mat()
        elif smoothing == 2:
            self.prob_mat = self.kneser_ney_mat()
        else:
            self.prob_mat = self.create_mat()

    def save(self, filename="bigram_mdl.pkl"):
        with open(filename, "wb") as f:
            pickle.dump(self, f)

    def get_emotion_score(self, size=-1, emotion=None):
        sntnc = self.genrate(emotion, size=size)
        emt_scr = emotion_scores(sntnc)
        emote = max(emt_scr, key=lambda x: x["score"])
        return sntnc, emote["label"], emote["score"]


def load(filename="bigram_mdl.pkl"):
    with open(filename, "rb") as f:
        obj = pickle.load(f)
    return obj
